In [ ]:
import numpy as np
from tqdm import tqdm
from pathlib import Path
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from lac.slam.semantic_feature_tracker import SemanticFeatureTracker
from lac.perception.segmentation import SemanticClasses
from lac.slam.frontend import Frontend
from lac.slam.backend import Backend
from lac.utils.plotting import plot_poses, plot_loop_closures, plot_3d_points, plot_semantic_points
from lac.util import load_data, load_stereo_images, load_images, positions_rmse_from_poses
from lac.utils.geometry import crop_points
from lac.params import SCENE_BBOX

%load_ext autoreload
%autoreload 2

# Load data


In [ ]:
# Load the data logs
data_path = "/home/shared/data_raw/LAC/runs/double_loop_preset1"
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)
print(f"Loaded {len(poses)} poses")

In [ ]:
# images = load_images(data_path, cameras=["FrontLeft", "FrontRight"], start_frame=0, end_frame=10000)
left_imgs, right_imgs = load_stereo_images(data_path, start_frame=0)
images = {"FrontLeft": left_imgs, "FrontRight": right_imgs}

# Initialize


In [ ]:
START_FRAME = 80
END_FRAME = len(poses) - 1

# TODO: we should probably keyframe the first frame (give it to the backend)

feature_tracker = SemanticFeatureTracker(cam_config)
frontend = Frontend(feature_tracker)
backend = Backend(poses[START_FRAME], feature_tracker)

frontend.initialize(left_imgs[START_FRAME], right_imgs[START_FRAME])

# Run


In [ ]:
eval_poses = [poses[START_FRAME]]

for frame in tqdm(range(START_FRAME + 2, END_FRAME, 2)):
    data = {
        "step": frame,
        "FrontLeft": left_imgs[frame],
        "FrontRight": right_imgs[frame],
        "imu": imu_data[frame],
    }
    data = frontend.process_frame(data)
    backend.update(data)
    eval_poses.append(poses[frame])

# Evaluate localization


In [ ]:
trajectory = backend.get_trajectory()

In [ ]:
fig = plot_poses(eval_poses, no_axes=True, color="black", name="Ground truth")
fig = plot_poses(trajectory, fig=fig, no_axes=True, color="orange", name="Backend poses")
fig = plot_loop_closures(trajectory, backend.loop_closures, fig=fig, showlegend=False)
fig.update_layout(height=900, width=1600, scene_aspectmode="data")
fig.show()

In [ ]:
print(f"RMSE: {positions_rmse_from_poses(eval_poses, trajectory)}")

# Evaluate mapping


In [ ]:
point_map = backend.project_point_map()

In [ ]:
map = np.load(
    "/home/shared/data_raw/LAC/heightmaps/competition/Moon_Map_01_preset_0.dat",
    allow_pickle=True,
)

In [ ]:
ground_points = points[labels == SemanticClasses.GROUND.value]
rock_points = points[labels == SemanticClasses.ROCK.value]

In [ ]:
from lac.mapping.mapper import bin_points_to_grid, interpolate_heights

In [ ]:
ground_grid = bin_points_to_grid(ground_points)
            map_array[:, :, 2] = ground_grid
            map_array[:] = interpolate_heights(map_array)

In [ ]:
points_cropped, keep_idxs = crop_points(points, SCENE_BBOX)
labels_cropped = labels[keep_idxs]

In [ ]:
trajectory = backend.get_trajectory()

In [ ]:
fig = plot_poses(eval_poses, no_axes=True, color="black", name="Ground truth")
fig = plot_poses(trajectory, fig=fig, no_axes=True, color="orange", name="Backend poses")
fig = plot_loop_closures(trajectory, backend.loop_closures, fig=fig, showlegend=False)
fig = plot_semantic_points(
    points_cropped[::10],
    labels_cropped[::10],
    names=[member.name for member in SemanticClasses],
    markersize=2,
    fig=fig,
)
fig.update_layout(height=900, width=1600, scene_aspectmode="data")
fig.show()

In [ ]:
fig.write_html(Path(data_path) / "loop_closures_1hz.html")

In [ ]:
positions_rmse_from_poses(eval_poses, trajectory)